In this notebook we will create a module to access the BLAST results in the **plants_vs_humans** database. We will use this module on Thursday to create a website that allows searching with a protein ID for similar sequences.

First we have to import the MySQLdb module

In [1]:
import MySQLdb

Next we need to define functions to open and close a connection to the database. The parameters are hardcoded here, why is that generally not a good idea?

In [2]:
def connect_db():
    """ Creates a connection to the plants_vs_humans database 
    Arguments:
        none
    Returns:
        database connection object
    """
    db = MySQLdb.connect(host="127.0.0.1", port=3306, user="root", passwd="pcb", 
                         db="plants_vs_humans")
    return db

def close_db(db):
    """ close the connection to the plants_vs_humans database 
    Arguments:
        db - database connection object
    Returns:
        Nothing
    """
    db.close()

Now we can make functions for specific tasks. The first function should return a list with all target IDs in the database. This will allow us to show a list with protein IDs on the web page for the user to choose from. The function connects to the database, gets a cursor to execute the SQL query and finally closes the database connection again.

In [54]:
def get_targets():
    """ retrieves all target IDs
    Arguments:
        None
    Returns:
        a list with all DISTINCT target IDs
    """
    
    db = connect_db()
    my_cursor = db.cursor()

    SQL = """SELECT DISTINCT target FROM blast_result"""
    num_res = my_cursor.execute(SQL)
    # print num_res # printing is just for development/debugging purposes
    
    targets = [] # this is where the target IDs should go
    for row in my_cursor:
        # process every result row 
        targets.append(row[0])
        # print row # printing is just for development/debugging purposes

    my_cursor.close()
    close_db(db)

    return targets

Ok, let's test the function

In [56]:
targets = get_targets()
print "There are %d targets"%len(targets)
print targets[0]

There are 2113 targets
sp|P62258|1433E_HUMAN


We now need a function to retrieve all the query proteins that have a match with a selected target protein ID. This function should allow filtering for results below a certain e-value. The result should include the query, the target, the evalue of the hit and the description of the query protein.

In [31]:
def get_rows_for_target(target_id,e_value=10.0):
    """ select the rows in the database that match the target_id
    Arguments:
        target_id - string with the ID of the target
        e_value - float representing the e_value threshold
    Returns:
        the resulting rows
    """

    SQL = """SELECT query,target,evalue,description
    	    FROM blast_result, protein
    	    WHERE blast_result.query = protein.ID
    	    AND blast_result.target LIKE '%%%s'
            AND blast_result.evalue <= 10.0;""" %target_id

    db = connect_db()
    my_cursor = db.cursor()
    num_res = my_cursor.execute(SQL)

    rows = my_cursor.fetchall()

    my_cursor.close()
    close_db(db)

    return rows

Let's test this function as well

In [32]:
rows = get_rows_for_target("1433E_HUMAN")
print "There are %d rows for target 1433E_HUMAN"%len(rows) 
print "This is the first row:",rows[0]

There are 60 rows for target 1433E_HUMAN
This is the first row: ('sp|P48347|14310_ARATH', 'sp|P62258|1433E_HUMAN', 2e-128, '14-3-3-like protein GF14 epsilon OS=Arabidopsis thaliana GN=GRF10 PE=2 SV=1')


Challenge: create a function that finds all queries with the same target and then creates one FASTA file with the sequences for these queries. The sequence lines should be limited to 60 characters, so a sequence with 80 characters should be split over two lines.

In [51]:
def get_queries_sequence_for_target_seq(target_id):
    """
    """
    SQL = """SELECT ID, sequence, query, target
	    FROM protein, blast_result
	    WHERE blast_result.query = protein.ID
	    AND blast_result.target LIKE '%%%s';"""%target_id

    db = connect_db()
    my_cursor = db.cursor()
    num_res = my_cursor.execute(SQL)

    fasta_filename = 'target_id.fasta'
    fasta_file = open(fasta_filename,'w')

    #insert magic here to write the FASTA file, remember: max 60 chars per line
    for row in my_cursor:
        print row
        header = '>' + row[0] + '\n'
        fasta_file.write(header)
        seq = row[1]
        if len(seq) <= 60:
            fasta_file.write(header)
        else:
            count = len(seq)//60 + 1
            start_index = 0
            end_index = 60
            for n in range(0,count):
                if n < count:
                    seq_line = seq[start_index:end_index] + '\n'
                    fasta_file.write(seq_line)
                if n == count:
                    seq_line = seq[start_index:len(seq)] + '\n'
                start_index += 60
                end_index += 60
                    
    
    fasta_file.close()
    my_cursor.close()
    close_db(db)

    return num_res

In [52]:
target_id = "1433E_HUMAN"
num_res = get_queries_sequence_for_target_seq(target_id)
print "Save %d sequences in the fasta file %s.fasta"%(num_res,target_id)

('sp|P48347|14310_ARATH', 'MENEREKQVYLAKLSEQTERYDEMVEAMKKVAQLDVELTVEERNLVSVGYKNVIGARRASWRILSSIEQKEESKGNDENVKRLKNYRKRVEDELAKVCNDILSVIDKHLIPSSNAVESTVFFYKMKGDYYRYLAEFSSGAERKEAADQSLEAYKAAVAAAENGLAPTHPVRLGLALNFSVFYYEILNSPESACQLAKQAFDDAIAELDSLNEESYKDSTLIMQLLRDNLTLWTSDLNEEGDERTKGADEPQDEN', 'sp|P48347|14310_ARATH', 'sp|P62258|1433E_HUMAN')
('sp|P93207|14310_SOLLC', 'MAALIPENLSREQCLYLAKLAEQAERYEEMVQFMDKLVLNSTPAGELTVEERNLLSVAYKNVIGSLRAAWRIVSSIEQKEESRKNEEHVHLVKEYRGKVENELSQVCAGILKLLESNLVPSATTSESKVFYLKMKGDYYRYLAEFKIGDERKQAAEDTMNSYKAAQEIALTDLPPTHPIRLGLALNFSVFYFEILNSSDKACSMAKQAFEEAIAELDTLGEESYKDSTLIMQLLRDNLTLWTSDAQDQLDES', 'sp|P93207|14310_SOLLC', 'sp|P62258|1433E_HUMAN')
('sp|Q9S9Z8|14311_ARATH', 'MENERAKQVYLAKLNEQAERYDEMVEAMKKVAALDVELTIEERNLLSVGYKNVIGARRASWRILSSIEQKEESKGNEQNAKRIKDYRTKVEEELSKICYDILAVIDKHLVPFATSGESTVFYYKMKGDYFRYLAEFKSGADREEAADLSLKAYEAATSSASTELSTTHPIRLGLALNFSVFYYEILNSPERACHLAKRAFDEAIAELDSLNEDSYKDSTLIMQLLRDNLTLWTSDLEEGGK', 'sp|Q9S9Z8|14311_ARATH', 'sp|P62258|1433E_HUMAN')
('sp|Q9C5W6|1

Now save the functions that you wrote to a file called db_functions.py and test it using the cell below

In [60]:
import db_functions

targets = db_functions.get_targets()
print targets[0]

OperationalError: (1065, 'Query was empty')